# Data Augmentation for brain tumor segmentation

## Qimin Zhang and Weiwei Qi

## 1. Config

In [18]:
import os
import random
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.optim as optim
from torch.utils.data import Dataset, SubsetRandomSampler
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF

from time import time

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import nibabel as nib
import tqdm

import torch.nn.functional as F
import torch.nn as nn

import skimage.transform

torch.manual_seed(4460)
np.random.seed(4460)

## 2. Convert Nifti to Numpy

In [17]:
dataPath = './data'


t1_path = os.path.join(dataPath, 'all_gbm_pre_reg')
t1_contrast_path = os.path.join(dataPath, 'all_gbm_post_reg')
tumor_path = os.path.join(dataPath, 'all_tumors_reg')
brain_mask_path = os.path.join(dataPath, 'brain_masks')


t1_path = os.path.join(dataPath, 'all_gbm_pre_reg')
t1_contrast_path = os.path.join(dataPath, 'all_gbm_post_reg')
tumor_path = os.path.join(dataPath, 'all_tumors_reg')
brain_mask_path = os.path.join(dataPath, 'brain_masks')


imgPaths = os.listdir(t1_path)
t1_locations = []
for path in imgPaths:
    if path.endswith('.nii.gz'):
        t1_locations.append(path)

imgPaths = os.listdir(t1_contrast_path)
t1_contrast_locations = []
for path in imgPaths:
    if path.endswith('.nii.gz'):
        t1_contrast_locations.append(path)

imgPaths = os.listdir(tumor_path)
tumor_locations = []
for path in imgPaths:
    if path.endswith('.nii.gz'):
        tumor_locations.append(path)
        
imgPaths = os.listdir(brain_mask_path)
brain_mask_locations = []
for path in imgPaths:
    if path.endswith('.nii.gz'):
        brain_mask_locations.append(path)
        
newPath = os.path.join(dataPath, 'numpyData')
if not os.path.exists(newPath):
    os.mkdir(newPath)
        
num_t1_path = os.path.join(newPath, 'num_t1')
if os.path.exists(num_t1_path):
    print('Folder already exists, so I did not create any numpy from t1.')
else:
    os.mkdir(num_t1_path)

    print('I am starting to convert t1 images.')
    path = t1_path
    t1_progress = tqdm.tqdm(enumerate(t1_locations))
    for i, imageLocation in t1_progress:
        t1_progress.set_description(f"Processing image {imageLocation}")
        # get the .nii image
        imageData = nib.load(os.path.join(path,imageLocation))
        # convert to numpy
        numpyImage = imageData.get_data()
        np.save(os.path.join(num_t1_path, imageLocation), numpyImage)
        
        
num_t1_contrast_path = os.path.join(newPath, 'num_t1_contrast')
if os.path.exists(num_t1_contrast_path):
    print('Folder already exists, so I did not create any numpy from t1 contrast.')
else:
    os.mkdir(num_t1_contrast_path)

    print('I am starting to convert t1 contrast images.')
    path = t1_contrast_path
    t1_contrast_progress = tqdm.tqdm(enumerate(t1_contrast_locations))
    for i, imageLocation in t1_contrast_progress:
        t1_contrast_progress.set_description(f"Processing image {imageLocation}")
        # get the .nii image
        imageData = nib.load(os.path.join(path,imageLocation))
        # convert to numpy
        numpyImage = imageData.get_data()
        np.save(os.path.join(num_t1_contrast_path, imageLocation), numpyImage)
        
num_tumor_path = os.path.join(newPath, 'num_tumor')
if os.path.exists(num_tumor_path):
    print('Folder already exists, so I did not create any numpy from tumors.')
else:
    os.mkdir(num_tumor_path)

    print('I am starting to convert tumor images.')
    path = tumor_path
    tumor_progress = tqdm.tqdm(enumerate(tumor_locations))
    for i, imageLocation in tumor_progress:
        tumor_progress.set_description(f"Processing image {imageLocation}")
        # get the .nii image
        imageData = nib.load(os.path.join(path,imageLocation))
        # convert to numpy
        numpyImage = imageData.get_data()
        np.save(os.path.join(num_tumor_path, imageLocation), numpyImage)
        
num_brain_mask_path = os.path.join(newPath, 'num_brain_mask')
if os.path.exists(num_brain_mask_path):
    print('Folder already exists, so I did not create any numpy from brain masks.')
else:
    os.mkdir(num_brain_mask_path)

    print('I am starting to convert brain mask images.')
    path = brain_mask_path
    brain_mask_progress = tqdm.tqdm(enumerate(brain_mask_locations))
    for i, imageLocation in brain_mask_progress:
        brain_mask_progress.set_description(f"Processing image {imageLocation}")
        # get the .nii image
        imageData = nib.load(os.path.join(path,imageLocation))
        # convert to numpy
        numpyImage = imageData.get_data()
        np.save(os.path.join(num_brain_mask_path, imageLocation), numpyImage)

print('All conversion done.')

Folder already exists, so I did not create any numpy from t1.
Folder already exists, so I did not create any numpy from t1 contrast.
Folder already exists, so I did not create any numpy from tumors.
Folder already exists, so I did not create any numpy from brain masks.
All conversion done.


## 3. Traditional Data Augmentation with TorchIO

In [16]:
import torchio
from torchio.transforms import Rescale, RandomAffine
from torchvision.transforms import Compose
from torch.utils.data import DataLoader



If you use TorchIO for your research, please cite the following paper:

Pérez-García et al., TorchIO: a Python library for efficient loading,
preprocessing, augmentation and patch-based sampling of medical images
in deep learning
(https://arxiv.org/abs/2003.04696)


see if I can commit: Mar 30